# Instalacion de dependencias
## En windows
```bash
pip install redis
```
## En Debian 
```bash
sudo apt install python3-redis
sudo apt install python3-psycopg2

```

In [ ]:
import redis
import psycopg2
import time

# Se configuran las cadenas de conexion
Dentro de este repositorio hay un docker compose, puede ejecutarlo y remplaza los valores al igual que hay un script de sql para crear la tabla de la prueba

In [ ]:
# Configuración del cliente de Redis
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0)

# Configuración de la conexión a PostgreSQL
conn = psycopg2.connect(dbname="Prueba", user="Usuario_Prueba", password="Password_Prueba", host="localhost")
cursor = conn.cursor()


# Función para obtener datos de PostgreSQL

In [ ]:
def fetch_from_database(key):
    print("Fetching from database...")
    cursor.execute("SELECT data FROM mytable WHERE id = %s", (key,))
    result = cursor.fetchone()
    if result:
        return result[0]
    return None

# Función para obtener datos utilizando Read Through Cache

In [ ]:

def get_data(key):
    # Intentar obtener los datos de la caché
    cached_value = redis_client.get(key)
    if cached_value:
        print("Cache hit!")
        return int(cached_value)
    else:
        print("Cache miss!")
        # Si no está en la caché, obtener los datos de la base de datos
        value = fetch_from_database(key)
        if value is not None:
            # Almacenar los datos en la caché con un tiempo de expiración de 10 segundos
            redis_client.setex(key, 10, value)
        return value

# Función para obtener datos utilizando Read Through Cache

In [ ]:
# Ejemplo de uso
if __name__ == "__main__":
    key = "1"
    # Primera lectura (esperamos una miss y una consulta a la base de datos)
    print(f"Data: {get_data(key)}")
    # Segunda lectura (esperamos un hit en la caché)
    print(f"Data: {get_data(key)}")
    # Esperar a que la caché expire
    time.sleep(11)
    # Tercera lectura (esperamos otra miss y una consulta a la base de datos)
    print(f"Data: {get_data(key)}")

# Cerrar conexión

In [ ]:
cursor.close()
conn.close()